In [ ]:
import os.path as op
import glob   
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import nibabel as nib 


from nilearn.image import load_img, math_img
from nilearn.plotting import plot_glass_brain, plot_design_matrix, plot_contrast_matrix, plot_stat_map, view_img, view_img_on_surf
from nilearn.glm.second_level import SecondLevelModel
from nilearn.glm.thresholding import threshold_stats_img
from nilearn.datasets import load_mni152_template, load_mni152_brain_mask
from nilearn.glm.second_level import non_parametric_inference
from nilearn.plotting import find_cut_slices

In [ ]:
#load the template that will be used
mni152_template = load_mni152_template()

In [ ]:
#list of of fileapths for files that will be used
effect_files = ["patient_X_path",
                "patient_Y_path"]

In [ ]:
#design matrix for one-sample t test
subjects = [f"sub{i+1:02}" for i in range(len(effect_files))]
design_matrix = pd.DataFrame([1]*len(subjects), index=subjects, columns=["swr_contrast"])

#fit second level model
swr_contrast_maps = [load_img(f) for f in effect_files]
second_level_model = SecondLevelModel()
second_level_model = second_level_model.fit(swr_contrast_maps, design_matrix=design_matrix)

#defining and computoing contrast
contrasts = {'High_vs_Low_SWR': [1]}
t_map = second_level_model.compute_contrast("swr_contrast", output_type="stat")
z_map = second_level_model.compute_contrast(contrasts['High_vs_Low_SWR'], output_type="z_score")
#multiple comparisson correction using FDR
thresholded_map, threshold = threshold_stats_img(
    z_map, 
    alpha=0.05, 
    height_control= "fdr", 
)

In [ ]:
#plot the maps
cut_coords = (X, Y, Z)

background_img = mni152_template
brain_mask = load_mni152_brain_mask()

#apply brain mask
t_map_masked = math_img("img1 * img2", img1=t_map, img2=brain_mask)
z_map_masked = math_img("img1 * img2", img1=z_map, img2=brain_mask)
thresholded_map_masked = math_img("img1 * img2", img1=thresholded_map, img2=brain_mask)


fig, axes = plt.subplots(2, 1, figsize=(12, 8), facecolor='white')

# Function to plot and label
def plot_panel(stat_img, threshold, ax, panel_label, title_text):
    display = plot_stat_map(
        stat_img,
        bg_img=background_img,
        threshold=threshold,
        display_mode='ortho',
        cut_coords=cut_coords,
        black_bg=False,
        axes=ax,
        colorbar=True
    )
    # Add panel label (top-left)
    ax.text(-0.05, 1.09, panel_label,  
            transform=ax.transAxes,
            fontdict=panel_font,
            va='top', ha='left')
    ax.set_title(title_text, fontdict=title_font, loc='center')

# A: T-map
plot_panel(t_map_masked, threshold=2.179, ax=axes[0], panel_label="A", title_text="T-map")

# B: Z-map
plot_panel(z_map_masked, threshold=1.96, ax=axes[1], panel_label="B", title_text="Z-map")

# C: FDR-corrected T-map
plot_panel(thresholded_map_masked, threshold=threshold, ax=axes[2],
           panel_label="B", title_text="FDR-corrected Z-map")



In [ ]:
#apply non-paramteric permutation based FWE correction for multiple comparisons

out_dict = non_parametric_inference(
    effect_files,  
    design_matrix=design_matrix,  
    second_level_contrast='swr_contrast',
    n_perm=10, 
    two_sided_test=True,
    threshold=0.1,
    n_jobs=-1
)

print(out_dict.keys())

In [ ]:
alpha = 0.05
masked = out_dict['logp_max_t'].get_fdata() > -np.log10(alpha)
masked_t_map = out_dict['t'].get_fdata() * masked

#masked t-map as a nifti
masked_t_map_img = nib.Nifti1Image(masked_t_map, out_dict['t'].affine)

In [ ]:
#get the smallest t-value for colorbar, the maps are alsready tresholded - to know id we can plot it
surviving_vals = masked_t_map[masked_t_map != 0]

if surviving_vals.size > 0:
    threshold_fwe = surviving_vals.min()
    print(f'FWE (perm.) p < 0.05 threshold: {threshold_fwe:.3f}')
else:
    threshold_fwe = None
    print('⚠️ No voxels survived FWE (perm.) correction at p < 0.05.')